In [ ]:
import pandas as pd
from itertools import combinations
import datetime


def abrir_kdd(ano, pchunksize = 250000, end=''):    
    end = f'C:\\Users\\letan\\PycharmProjects\\KDD_partos\\KDD{ano}.csv'
    # divisão em pedaços para depois concatenar, em função da limitação de RAM
    chunk = pd.read_csv(end, sep=',', engine='python', chunksize=pchunksize ) 
    df = pd.concat(chunk)
    df = df.drop(columns = 'Unnamed: 0')
    cats = ['def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_gestacao', 'def_gravidez', 'def_parto', 'def_consultas',
            'dia_semana_nasc', 'def_sexo', 'def_raca_cor', 'RACACORMAE', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO', 
            'TPNASCASSI', 'STDNEPIDEM', 'TPROBSON', 'nasc_AMAZONIA', 'nasc_FRONTEIRA', 'nasc_CAPITAL', 'res_AMAZONIA',
            'res_FRONTEIRA', 'res_CAPITAL', 'nasc_REGIAO', 'res_REGIAO', 'def_parto_prematuro', 'fx_mae', 'fx_pai',
            'fx_apgar1', 'fx_apgar5', 'fx_peso', 'Cesárea Anterior', 'Parto Normal Anterior', 'Gestação Anterior',
            'Filho Morto Anterior', 'mes_nasc', 'turno_nasc', 'GGOCBO', 'nasc_SIGLA_UF', 'res_SIGLA_UF', 'fx_mes_prenat']
    for categorico in cats:
        df[categorico] = df[categorico].astype("category")
    nums = ['CODANOMAL']
    return df
    

def regras_ano(df, atributo, kitem, pconf = 0.8, psup = 0.2):
    print(f'Ajustes: Atrubuto alvo - "{atributo}". Conjuntos de {kitem} atributos. Confiança de {pconf} e suporte de {psup}')
    comeco = datetime.datetime.now()
    print(f'Início: {comeco}\n')
    #fazer uma lista com as colunas do dataframe para combiná-las. Remover o atributo alvo, para evitar regras 'Se A, então A'
    colunas = list(df.columns)
    colunas.remove(atributo)
    regras = set()
    for k in range(1,kitem):
        try:
            for attr in combinations(colunas, k):                                          # k-itemsets
                print(f'Combinação: {attr}')
                itemsets_sup = df.value_counts(list(attr), normalize = True)               # contagem de casos (sem o alvo)
                itemsets_desc = itemsets_sup.index                                         # relação dos attr por valor dos atributos
                regras_sup  = df.value_counts([atributo] + list(attr), normalize = True)   # contagem da regra (com o alvo)
                for i in range(len(itemsets_sup)):
                    for valor in df[atributo].unique():
                        descr_ant = f'{list(attr)}, {itemsets_desc[i]}'                    # cálculo para cada k-upla de valores
                        sup_ant = itemsets_sup[i]                                          # suporte do antecedente
                        if sup_ant > psup:
                            sup_reg = regras_sup[tuple([valor] + list(itemsets_desc[i]))]  # suporte da regra
                            confianca = round(sup_reg/sup_ant, 3)                          # confiança
                            if confianca > pconf:                                                       
                                regra = (attr, itemsets_desc[i], (atributo, valor), confianca, sup_reg)
                                regras.add(regra)
        except KeyError:
            continue
    print(f'{len(regras)} regra(s) encontrada(s)')
    final = datetime.datetime.now()
    print(f'Fim: {final}\n')
    tempo =  final - comeco
    linhas = df.shape[0]
    print(f'Tempo para {linhas} linhas: {tempo}')       
    return regras

def condensar(cj_regras):
    lr = list(cj_regras)
    for item1 in lr:
        for item2 in lr:
            if  set(item2[0]) < (set(item1[0])) and set(item2[1]) < (set(item1[1])):
                lr.remove(item2)
    print(f'Condensado para {len(lr)} regras')
                
    return set(lr)
    
def impr_regras(cjregras):
    cont = 0
    for regra in cjregras:
        cont += 1
        txt_regra = f'{cont}: '
        for i in range(len(regra[0])):
            txt_regra += f'Se {regra[0][i]} = {regra[1][i]}, '
        print(f'{txt_regra}então {regra[2][0]} = {regra[2][1]} (confiança {regra[3]: .2%}, suporte {regra[4]: .2%})\n')
            
        

In [21]:
a = abrir_kdd(2017)

In [22]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2923510 entries, 0 to 2923509
Data columns (total 46 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   def_loc_nasc           category
 1   def_est_civil          category
 2   ESCMAE                 category
 3   def_gestacao           category
 4   def_gravidez           category
 5   def_parto              category
 6   def_consultas          category
 7   ano_nasc               int64   
 8   dia_semana_nasc        category
 9   def_sexo               category
 10  def_raca_cor           category
 11  def_anomalia           object  
 12  CODANOMAL              object  
 13  RACACORMAE             category
 14  TPAPRESENT             category
 15  STTRABPART             category
 16  STCESPARTO             category
 17  TPNASCASSI             category
 18  STDNEPIDEM             category
 19  CODPAISRES             float64 
 20  TPROBSON               category
 21  nasc_AMAZONIA          category

In [ ]:
b = regras_ano(a, 'def_parto', 6)
c = condensar(b)
impr_regras(c)
a.value_counts('def_loc_nasc')

Início: 2021-07-11 21:36:34.601191

Combinação: ('def_loc_nasc',)
Combinação: ('def_est_civil',)
Combinação: ('ESCMAE',)
Combinação: ('def_gestacao',)
Combinação: ('def_gravidez',)
Combinação: ('def_consultas',)
Combinação: ('ano_nasc',)
Combinação: ('def_loc_nasc', 'def_est_civil')
Combinação: ('def_loc_nasc', 'ESCMAE')
Combinação: ('def_loc_nasc', 'def_gestacao')
Combinação: ('def_loc_nasc', 'def_gravidez')
Combinação: ('def_loc_nasc', 'def_consultas')
Combinação: ('def_loc_nasc', 'ano_nasc')
Combinação: ('def_loc_nasc', 'dia_semana_nasc')
Combinação: ('def_loc_nasc', 'def_sexo')
Combinação: ('def_loc_nasc', 'def_raca_cor')
Combinação: ('def_loc_nasc', 'def_anomalia')
Combinação: ('def_loc_nasc', 'CODANOMAL')
Combinação: ('def_loc_nasc', 'RACACORMAE')
Combinação: ('def_loc_nasc', 'TPAPRESENT')
Combinação: ('def_loc_nasc', 'STTRABPART')
Combinação: ('def_loc_nasc', 'STCESPARTO')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE')
Combinação: ('def_loc_nasc', 'def_est_civil', 'def_

Combinação: ('def_loc_nasc', 'def_est_civil', 'def_gravidez', 'def_raca_cor')
Combinação: ('def_loc_nasc', 'def_est_civil', 'def_gravidez', 'def_anomalia')
Combinação: ('def_loc_nasc', 'def_est_civil', 'def_gravidez', 'CODANOMAL')
Combinação: ('def_loc_nasc', 'def_est_civil', 'def_gravidez', 'RACACORMAE')
Combinação: ('def_loc_nasc', 'def_est_civil', 'def_gravidez', 'TPAPRESENT')
Combinação: ('def_loc_nasc', 'def_est_civil', 'def_gravidez', 'STTRABPART')
Combinação: ('def_loc_nasc', 'def_est_civil', 'def_gravidez', 'STCESPARTO')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_gestacao', 'def_gravidez')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_gestacao', 'def_consultas')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_gestacao', 'ano_nasc')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_gestacao', 'dia_semana_nasc')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_gestacao', 'def_sexo')
Combinação: ('def_loc_nasc', 

Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'TPAPRESENT')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'STTRABPART')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'STCESPARTO')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'TPNASCASSI')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'STDNEPIDEM')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'CODPAISRES')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'TPROBSON')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'nasc_AMAZONIA')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'nasc_FRONTEIRA')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'nasc_CAPITAL')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_consultas', 'res_AMAZONIA')
Combinação: ('def_loc_nasc', 'def_

Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'fx_pai')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'fx_apgar1')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'fx_apgar5')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'fx_mes_prenat')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'fx_peso')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'Cesárea Anterior')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'Parto Normal Anterior')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'Gestação Anterior')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'Filho Morto Anterior')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'mes_nasc')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'dia_semana_nasc', 'turno_na

Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'res_AMAZONIA')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'res_FRONTEIRA')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'res_CAPITAL')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'nasc_SIGLA_UF')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'res_SIGLA_UF')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'nasc_REGIAO')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'res_REGIAO')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'codanomal_categoria')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'def_parto_prematuro')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'fx_mae')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'def_anomalia', 'fx_pai')
Combinação: ('def_loc_nasc', 'de

Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'res_AMAZONIA')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'res_FRONTEIRA')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'res_CAPITAL')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'nasc_SIGLA_UF')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'res_SIGLA_UF')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'nasc_REGIAO')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'res_REGIAO')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'codanomal_categoria')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'def_parto_prematuro')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'fx_mae')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE', 'TPAPRESENT', 'fx_pai')
Combinação: ('def_loc_nasc', 'def_est_civil', 'ESCMAE'

In [ ]:
n2017 = a[a['nasc_REGIAO'] == 'Norte']

nr2017 = regras_ano(n2017, 'def_parto', 6)

In [ ]:
nc2017 = condensar(nr2017)
impr_regras(nc2017)
n2017.value_counts('def_loc_nasc')

In [ ]:
del n2017
ne2017 = a[a['nasc_REGIAO'] == 'Nordeste']
ner2017 = regras_ano(ne2017, 'def_parto', 6)

In [ ]:
impr_regras(ner2017)

In [ ]:
del ne2017
co2017 = a[a['nasc_REGIAO'] == 'Centro-Oeste']
cor2017 = regras_ano(co2017, 'def_parto', 6)

In [ ]:
del co2017
se2017 = a[a['nasc_REGIAO'] == 'Sudeste']
ser2017 = regras_ano(se2017, 'def_parto', 6)

In [ ]:
sec2017 = condensar(ser2017)
impr_regras(sec2017)

In [ ]:
del se2017
s2017 = a[a['nasc_REGIAO'] == 'Sul']
sr2017 = regras_ano(s2017, 'def_parto', 6)

In [ ]:
sc2017 = condensar(sr2017)
impr_regras(sc2017)

In [ ]:
parto_esc = a.value_counts(['def_parto', 'ESCMAE'], normalize = True)
print(parto_esc)
print(f'Normal com 12+ anos de estudo: {parto_esc[( "Vaginal",   "12 e mais")]: .2%}')
print(f"Cesáreo com 12+ anos de estudo: {parto_esc[( 'Cesáreo',   '12 e mais')]: .2%}")
print(f'Normal com 8 a 11 anos de estudo: {parto_esc[( "Vaginal",   "8 a 11 anos")]: .2%}')
print(f"Cesáreo com 8 a 11 anos de estudo: {parto_esc[( 'Cesáreo',   '8 a 11 anos')]: .2%}")
mincesareo = parto_esc[( 'Cesáreo',   '1 a 3 anos')] + parto_esc[( 'Cesáreo',   '4 a 7 anos')]
minnormal =  parto_esc[( "Vaginal",   "1 a 3 anos")] + parto_esc[( "Vaginal",   "4 a 7 anos")]
print(f'Normal com menos de 8 anos de estudo: {minnormal: .2%}')
print(f"Cesáreo com  menos de 8 anos de estudo: {mincesareo: .2%}")

In [ ]:
parto_dom = a[a['def_loc_nasc'] == 'Domicílio']
graf_pdom = parto_dom.value_counts(['nasc_REGIAO'], normalize = True, sort=True)
print(graf_pdom)
graf_pdom.plot.pie(ylabel = "Partos Domiciliares", autopct='%1.1f%%')

In [ ]:
parto_loc = a.value_counts(['def_loc_nasc', 'def_parto'])
print(parto_loc)